In [4]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

    Total Debt as % of GDP << ................=> 1966 by Q1 to Q4 https://fred.stlouisfed.org/series/GFDEGDQ188S
    Federal Funds Rate + .....................=> 1954-07-01
    Industrial Production Index (Indpro) .....=> 1919-01-01 https://fred.stlouisfed.org/series/INDPRO
    Non-farm payroll .........................=> 1939-01-01 https://fred.stlouisfed.org/series/PAYEMS
    Unemployment Rate + ......................=> 1955-01-01
    SPY500 + .................................=> 1922 https://www.investing.com/indices/us-spx-500-historical-data
    10 year Treasury Bond -/+ ................=> 1962 https://www.macrotrends.net/2016/10-year-treasury-bond-rate-yield-chart
    Slope of the yield curve -/+ .............=> 1962 https://fred.stlouisfed.org/series/T10YFF
    CPI ......................................=> 1947-01-01
    Петролен скок: Скокът на цените на суровия петрол, който се отразява на джоба на всяко домакинство, е един от малкото наистина надеждни индикатори за предстоящ спад. Цените на петрола се повишиха с близо $25 от най-ниските си стойности през лятото и надхвърлиха $95 за барел.
https://fred.stlouisfed.org/series/WTISPLC
    
    Глобален спад: Останалата част от света може да повлече САЩ надолу. Втората по големина икономика, Китай, е затънала в криза с недвижимите имоти. В еврозоната кредитирането се свива с по-бързи темпове, отколкото в разгара на кризата с държавния дълг – знак, че вече стагниращият растеж ще се понижи. >> Global GDP > Years only!

On Macrotrends only data starting  in1962 was available. I will therefore only consider data starting in 196
Define the data set on which to perform exploratory data analysis (August 1955 to January 1972) to ensure no intersection with cross-validation periods.
I would have loved to use a credit spread index of some type (investment grade or high yield) here, but could not find a credit spread index with data going back to 1955.
America's national debt in dollars is generally viewed as less important than its proportion to the country’s gross domestic product (GDP) or the debt-to-GDP ratio because a country’s tax base grows alongside its economy. It increases revenue that the government can raise to service the debt. The U.S. national debt-to-GDP was 122.34% at the close of the first quarter of 2024..


## Poject: <a style="color:purple;">**Macroeconomic Analysis of the US Economy**</a> - part 2

### Author: @**engine**, Date: хх.хх.2024

### Abstract:
**TODO**

### 1. **Defining the problem and main assumptions**

In the **first part** of this study, looking at the distribution of funds from the US budget, we did not see good prospects in the future. It is not invested in the development of the USA, but in the wars, their consequences and the interest on the loans taken.
The share of interest payments on borrowed loans is also very large. The almost direct linear dependence of the expansion of the economy on inflation (printing money) and borrowing was also shown. The latter, since the dollar is a world currency, means printing money again. As a result of all that has been said so far, we can safely summarize that the American economy is addicted to printing new money. The trade balance is strongly negative with a worsening trend. Science is not a priority. If for some reason the printing of new money stops and/or the dollars of the world go "home" to the USA, we are waiting for "Great Depression - 2".

From here, we can ask ourselves the following question: <a style="color:purple;">**Can we use ML to predict approximately when the US economy will go into recession?**</a><br>The **second part** of the present study will be based on the answer to this question.

There are **2 main theoretical assumptions** at play:<br>l.


### 2. **Preparing the Data**

#### 2.1 Criteria according to which the data for the present study were collected.

**TODO**

#### 2.2 GROSS DOMESTIC PRODUCT (GDP)

From this site **[FEDERAL RESERVE BANK of ST. LOUIS](https://fred.stlouisfed.org/series/GDP)** we only download information for the period of our study: **Gross Domestic Product (GDP)**. [3]
Units: <a style="color:purple;">**Billions of Dollars**</a>, Seasonally Adjusted Annual Rate.
Frequency: Quarterly.

In [5]:
gdp = pd.read_csv('data/GDP.csv') # Read the CSV file
gdp.columns = ["date", "gdp"]
gdp['gdp'] = gdp['gdp'] * 1_000_000_000 # convert to dollars
gdp # Show all data.
# __________ DataFrame (DF) with real data! __________

,date,gdp
0,2022-07-01,2.599464e+13
1,2022-10-01,2.640840e+13
2,2023-01-01,2.681360e+13
3,2023-04-01,2.706301e+13
4,2023-07-01,2.761013e+13
5,2023-10-01,2.795700e+13
6,2024-01-01,2.826917e+13
7,2024-04-01,2.862915e+13


#### 2.3 CONSUMER PRICE INDEX (CPI)

From this site **[FEDERAL RESERVE BANK of ST. LOUIS](https://fred.stlouisfed.org/series/CPIAUCSL)** we only download information for the period of our study:
**Consumer Price Index (CPI)** for All Urban Consumers, All Items in U.S. City Average (CPIAUCSL). [4]

In [11]:
cpi = pd.read_csv('data/Consumer Price Index.csv') # Read the CSV file
cpi.columns = ["date", "cpi"]
cpi.head() # Show first rows only...

,date,cpi
0,2022-07-01,294.977
1,2022-08-01,295.209
2,2022-09-01,296.341
3,2022-10-01,297.863
4,2022-11-01,298.648


#### 2.4 CONSUMER CONFIDENCE INDEX (CCI)

From this site **[MacroVar](https://macrovar.com/united-states/consumer-confidence-index/)** we download information: **CONSUMER CONFIDENCE INDEX (CCI)**, United States Conference Board Consumer Confidence Analytics & Data. [5]
Since the data is from 2014, we filter only those that are for our studied period.

In [15]:
# NO PERIOD SET !!! ----------------------------------------------------------------------- !!!!!!
cci = pd.read_csv('data/CCI.csv') # Read the CSV file
cci.columns = ["date", "cci"]
# Convert the "date" column to datetime format
cci['date'] = pd.to_datetime(cci['date'])
# Filter the data for the period from 2022 to 2024
filtered_cci = cci[(cci['date'] >= '2022-06-01') & (cci['date'] <= '2024-05-31')]
del cci
filtered_cci = filtered_cci.reset_index(drop=True)
filtered_cci.head() # Show first rows only...

,date,cci
0,2022-06-30,98.7
1,2022-07-31,95.3
2,2022-08-31,103.6
3,2022-09-30,107.8
4,2022-10-31,102.5


#### 2.5 INTEREST RATES

From this site **[FEDERAL RESERVE BANK of ST. LOUIS](https://fred.stlouisfed.org/series/FEDFUNDS)** we only download information for the period of our study: **Federal Funds Effective Rate (FEDFUNDS)**. [6] The federal funds rate is the interest rate at which depository institutions trade federal funds (balances held at Federal Reserve Banks) with each other overnight. When a depository institution has surplus balances in its reserve account, it lends to other banks in need of larger balances. In simpler terms, a bank with excess cash, which is often referred to as liquidity, will lend to another bank that needs to quickly raise liquidity.
 1. The rate that the borrowing institution pays to the lending institution is determined between the two banks; the weighted average rate for all of these types of negotiations is called the effective federal funds rate.
 2. The effective federal funds rate is essentially determined by the market but is influenced by the Federal Reserve through open market operations to reach the federal funds rate target.

The Federal Open Market Committee (FOMC) meets eight times a year to determine the federal funds target rate. As previously stated, this rate influences the effective federal funds rate through open market operations or by buying and selling of government bonds (government debt).

In [19]:
fed_funds = pd.read_csv('data/FEDFUNDS.csv') # Read the CSV file
fed_funds.columns = ["date", "fed_funds"]
fed_funds.head()
# ____ Display data in percentages!! _________ Show first rows only... !!!

,date,fed_funds
0,2022-07-01,1.68
1,2022-08-01,2.33
2,2022-09-01,2.56
3,2022-10-01,3.08
4,2022-11-01,3.78


#### 2.6 TRADE BALANCE

From this site **[FEDERAL RESERVE BANK of ST. LOUIS](https://fred.stlouisfed.org/series/BOPGTB)** we only download information for the period of our study:
**Trade Balance: Goods, Balance of Payments Basis (BOPGTB)**,
Units: <a style="color:purple;">**Millions of Dollars**</a>, Seasonally Adjusted. [7]

In [23]:
# NO PERIOD SET !!! ----------------------------------------------------------------------- !!!!!!
tb = pd.read_csv('data/BOPGTB.csv') # Read the CSV file
tb.columns = ["date", "trade_balance"]

tb['trade_balance'] = tb['trade_balance'] * 1_000_000 # Convert to dollars!!
tb.head()
# __________ DF with real data! __________ Show first rows only...

,date,trade_balance
0,2022-05-01,-1.036630e+11
1,2022-06-01,-9.973500e+10
2,2022-07-01,-8.923600e+10
3,2022-08-01,-8.678200e+10
4,2022-09-01,-9.001500e+10


#### 2.7 US DEBT

From this site **[FiscalData](https://fiscaldata.treasury.gov/datasets/monthly-statement-public-debt/summary-of-treasury-securities-outstanding)** we download information: **U.S. Treasury Monthly Statement of the Public Debt (MSPD)**, Date Range (Record Date): 5 Years. <a style="color:purple;">**Debt in Millions of Dollars!**</a> [8]

The U.S. Treasury Monthly Statement of the Public Debt (MSPD) dataset details the Treasury's outstanding debts and the statutory debt limit. Debt is categorized by whether it is marketable or non-marketable and whether it is debt held by the public or debt held by government agencies. All amounts are reported in millions of U.S. dollars. Data is published on the fourth business day of each month, detailing the debt as of the end of the previous month.

In [27]:
# read the dataset using the compression zip
debt = pd.read_csv('data/MSPD_SumSecty.zip',compression='zip')

# We leave only the requested columns...
keep_col = ['Record Date', 'Security Type Description', 'Total Public Debt Outstanding (in Millions)']
debt_filtered = debt[keep_col]

# We leave only the requested dates...
debt_filtered = debt_filtered[(debt_filtered['Record Date'] >= '2022-07-01') & (debt_filtered['Record Date'] <= '2024-06-31')]

# selecting rows based on condition
debt_filtered = debt_filtered.loc[debt_filtered['Security Type Description'] == 'Total Public Debt Outstanding']

# rename columns
debt_filtered.columns = ["date", "type", 'total_debt']

debt_filtered.head() # Show first rows only...

,date,type,total_debt
13,2024-06-30,Total Public Debt Outstanding,3.483163e+07
27,2024-05-31,Total Public Debt Outstanding,3.466712e+07
41,2024-04-30,Total Public Debt Outstanding,3.461699e+07
55,2024-03-31,Total Public Debt Outstanding,3.458653e+07
69,2024-02-29,Total Public Debt Outstanding,3.447108e+07


#### 2.8 UNEMPLOYMENT

From this site **[FEDERAL RESERVE BANK of ST. LOUIS](https://fred.stlouisfed.org/series/LRHUTTTTUSM156S)** we only download information for the period of our study:  Infra-Annual Labor Statistics: **Monthly Unemployment Rate**. [9] Units: <a style="color:purple;">**Percent**</a>, Seasonally Adjusted. Frequency: Monthly. 

In [33]:
unemployment = pd.read_csv('data/LRHUTTTTUSM.csv')
unemployment.columns = ["date", "unemployment"]
unemployment.head()
# ____ Display data in percentages!! ______ Show first rows only...

,date,unemployment
0,2022-07-01,3.5
1,2022-08-01,3.6
2,2022-09-01,3.5
3,2022-10-01,3.6
4,2022-11-01,3.6


### 5. **Conclusions**

1. **TODO**

### Resources:
1. **Recession Prediction using Machine Learning** https://towardsdatascience.com/recession-prediction-using-machine-learning-de6eee16ca94
2. **...**
3. **...**
4. **...**
5. **...**
6. **...**
7. **...**
8. **...**
9. **...**
10. **...**
11. **...**
12. **...**
13. **...**
14. **...**
15. **...**